In [2]:
#################################################
#created the 26/04/2018 13:42 by Alexis Blanchet#
#################################################


import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import scipy.stats
import matplotlib
import plotly.offline as offline
import plotly.graph_objs as go

In [4]:
CHAINE = '192'
DATE = '2017-12-01'
JOINDATE = "".join(list(DATE.split('-')))

c = list(CHAINE)
n = 4-len(c)
c = ['0']*n
a = "".join(c)
CHAINE2 = a+CHAINE

In [9]:
X_PTV = pd.read_csv('PTV/IPTV_'+CHAINE2+'_'+DATE+'_TF1.csv',index_col=False).drop(['Unnamed: 0'],axis=1)
X_RTS = pd.read_csv('RTS/sfrdaily_'+JOINDATE+'_0_'+CHAINE+'_0_cleandata-processed.csv',index_col=False)
y = pd.read_csv('RTS/sfrdaily_'+JOINDATE+'_0_'+CHAINE+'_0_cleandata-y.csv',index_col=False)

In [10]:
X_PTV.head()

,DOLBY,DERDIF,DOLBY51,CLE-FORMAT,description programme,DIRECT,CSA,@CLEEMI,INEDIT_EN_CLAIR,RESUME,...,DUREE,VOST,PREMDIF,STEREO,NOUVEAUTE,TITRE,INEDIT_CRYPTE,DATE,HEURE,minutes
0,0,0,0,12,Magazine jeunesse,0,1,122835453,1,0,...,120,0,0,1,0,TFou,0,2017-12-01,06:25:00,385
1,0,0,0,16,Météo,0,1,122842665,0,0,...,5,0,0,1,0,Météo,0,2017-12-01,08:25:00,505
2,0,0,0,12,Magazine de télé-achat,0,1,122842666,0,0,...,45,0,0,1,0,Téléshopping,0,2017-12-01,08:30:00,510
3,0,0,0,16,Météo,0,1,122842667,0,0,...,5,0,0,1,0,Météo,0,2017-12-01,09:15:00,555
4,0,0,0,3,Série réaliste,0,1,116093509,0,0,...,30,0,0,1,0,Petits secrets entre voisins,0,2017-12-01,09:20:00,560


In [7]:
X_RTS.head()


,t-3,t-2,t-1,t,minutes,diff t t-1,diff t t-2,diff t t-3,diff t-1 t-2,diff t-1 t-3,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label
0,3148165.0,3159788.0,3152419.0,3157207,3,4788.0,-2581.0,9042.0,-7369.0,4254.0,11623.0,3154394.75,2812.25,-1290.5,3014.000000,2127.0,1.865585e-13,0.5,2.905194e-35,-0.0
1,3159788.0,3152419.0,3157207.0,3136355,4,-20852.0,-16064.0,-23433.0,4788.0,-2581.0,-7369.0,3151442.25,-15087.25,-8032.0,-7811.000000,-1290.5,1.865585e-13,0.5,6.906554e-34,-0.0
2,3152419.0,3157207.0,3136355.0,3123438,5,-12917.0,-33769.0,-28981.0,-20852.0,-16064.0,4788.0,3142354.75,-18916.75,-16884.5,-9660.333333,-8032.0,1.865585e-13,0.5,2.250786e-33,-0.0
3,3157207.0,3136355.0,3123438.0,3147105,6,23667.0,10750.0,-10102.0,-12917.0,-33769.0,-20852.0,3141026.25,6078.75,5375.0,-3367.333333,-16884.5,1.865585e-13,0.5,1.985593e-33,-0.0
4,3136355.0,3123438.0,3147105.0,3148270,7,1165.0,24832.0,11915.0,23667.0,10750.0,-12917.0,3138792.00,9478.00,12416.0,3971.666667,5375.0,1.865585e-13,0.5,8.184194e-34,0.0


In [8]:
X_RTS.shape

(1437, 20)

In [11]:
tdf = pd.DataFrame()
tdf['debut'] = X_PTV['minutes']
tdf['fin'] = tdf['debut']+X_PTV['DUREE']
tdf['titre'] = X_PTV['TITRE']
tdf.head()

,debut,fin,titre
0,385,505,TFou
1,505,510,Météo
2,510,555,Téléshopping
3,555,560,Météo
4,560,590,Petits secrets entre voisins


In [61]:
df = pd.DataFrame()
df['minutes'] = X_RTS['minutes']
df['isinprogramme'] = 0
df['TITRE'] = '0'
df['debut'] = 0
df['fin'] = 0

df.head()
    


,minutes,isinprogramme,TITRE,debut,fin
0,3,0,0,0,0
1,4,0,0,0,0
2,5,0,0,0,0
3,6,0,0,0,0
4,7,0,0,0,0


In [62]:
for index, row in df.iterrows():
    for index2, row2 in tdf.iterrows():
        if(int(row2['debut']) <= int(row['minutes']) <= int(row2['fin'])):
            df.set_value(index, 'isinprogramme', 1)
            df.set_value(index, 'fin', row2['fin'])
            df.set_value(index, 'debut', int(row2['debut']))
            df.ix[index,'TITRE']=row2['titre']
            break
        else:
            pass


In [63]:
df = pd.merge(df, X_RTS, on='minutes', how='left')

In [64]:
print(df.shape)
df.head()

(1437, 24)


,minutes,isinprogramme,TITRE,debut,fin,t-3,t-2,t-1,t,diff t t-1,...,diff t-2 t-3,mean,distance to mean,pente t t-2,pente t t-3,pente t-1 t-3,probability,distribution,skewness,label
0,3,0,0,0,0,3148165.0,3159788.0,3152419.0,3157207,4788.0,...,11623.0,3154394.75,2812.25,-1290.5,3014.000000,2127.0,1.865585e-13,0.5,2.905194e-35,-0.0
1,4,0,0,0,0,3159788.0,3152419.0,3157207.0,3136355,-20852.0,...,-7369.0,3151442.25,-15087.25,-8032.0,-7811.000000,-1290.5,1.865585e-13,0.5,6.906554e-34,-0.0
2,5,0,0,0,0,3152419.0,3157207.0,3136355.0,3123438,-12917.0,...,4788.0,3142354.75,-18916.75,-16884.5,-9660.333333,-8032.0,1.865585e-13,0.5,2.250786e-33,-0.0
3,6,0,0,0,0,3157207.0,3136355.0,3123438.0,3147105,23667.0,...,-20852.0,3141026.25,6078.75,5375.0,-3367.333333,-16884.5,1.865585e-13,0.5,1.985593e-33,-0.0
4,7,0,0,0,0,3136355.0,3123438.0,3147105.0,3148270,1165.0,...,-12917.0,3138792.00,9478.00,12416.0,3971.666667,5375.0,1.865585e-13,0.5,8.184194e-34,0.0


In [65]:
df = pd.merge(df, X_PTV, on='TITRE', how='left')

In [67]:
print(df.shape)
df[500:600]

(1792, 62)


,minutes_x,isinprogramme,TITRE,debut,fin,t-3,t-2,t-1,t,diff t t-1,...,GENRE,DUREE,VOST,PREMDIF,STEREO,NOUVEAUTE,INEDIT_CRYPTE,DATE,HEURE,minutes_y
500,503,1,TFou,385,505,58226416.0,58527937.0,58809492.0,59047300,237808.0,...,"OrderedDict([('@CLE', '403'), ('@GENRESIMPLE',...",120.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,06:25:00,385.0
501,504,1,TFou,385,505,58527937.0,58809492.0,59047300.0,59295724,248424.0,...,"OrderedDict([('@CLE', '403'), ('@GENRESIMPLE',...",120.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,06:25:00,385.0
502,505,1,TFou,385,505,58809492.0,59047300.0,59295724.0,59652328,356604.0,...,"OrderedDict([('@CLE', '403'), ('@GENRESIMPLE',...",120.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,06:25:00,385.0
503,506,1,Météo,505,510,59047300.0,59295724.0,59652328.0,59840260,187932.0,...,"OrderedDict([('@CLE', '951'), ('@GENRESIMPLE',...",5.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,08:25:00,505.0
504,506,1,Météo,505,510,59047300.0,59295724.0,59652328.0,59840260,187932.0,...,"OrderedDict([('@CLE', '951'), ('@GENRESIMPLE',...",5.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,09:15:00,555.0
505,506,1,Météo,505,510,59047300.0,59295724.0,59652328.0,59840260,187932.0,...,"OrderedDict([('@CLE', '951'), ('@GENRESIMPLE',...",15.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,13:40:00,820.0
506,506,1,Météo,505,510,59047300.0,59295724.0,59652328.0,59840260,187932.0,...,"OrderedDict([('@CLE', '951'), ('@GENRESIMPLE',...",5.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,19:55:00,1195.0
507,506,1,Météo,505,510,59047300.0,59295724.0,59652328.0,59840260,187932.0,...,"OrderedDict([('@CLE', '951'), ('@GENRESIMPLE',...",5.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,20:40:00,1240.0
508,507,1,Météo,505,510,59295724.0,59652328.0,59840260.0,60089019,248759.0,...,"OrderedDict([('@CLE', '951'), ('@GENRESIMPLE',...",5.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,08:25:00,505.0
509,507,1,Météo,505,510,59295724.0,59652328.0,59840260.0,60089019,248759.0,...,"OrderedDict([('@CLE', '951'), ('@GENRESIMPLE',...",5.0,0.0,0.0,1.0,0.0,0.0,2017-12-01,09:15:00,555.0
